In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import io
# from sklearn.datasets import load_breast_cancer
import redis
r = redis.Redis()


from flask import Flask, request, render_template, session, redirect

import time
import datetime
from datetime import timedelta

In [110]:
!pip install -U flask-cors

  Using cached Flask_Cors-3.0.10-py2.py3-none-any.whl (14 kB)


In [235]:
def count_last_hour_unique_hashtags():
    current_ts = time.time()
    return r.zcount('last_hour_tags', current_ts-3600, current_ts)
    

In [238]:
count_last_hour_unique_hashtags()

15

In [186]:
def get_delta(d1, d2):
    delta = d2 - d1
    return delta
def count_twitts_by_range(start_date,end_date):
    delta = get_delta(start_date,end_date)
    total_count = 0
    h_diff = int(delta.total_seconds()/3600)
    for i in range(delta.days * 24 + h_diff):
        date_key = start_date + timedelta (hours=i)
        tmp = r.get(extract_date_key(date_key))
#         print(date_str)
        if tmp != None:
            total_count += int(tmp.decode('ascii'))
#         print (extract_date_key(date_str))
    return total_count
def extract_date_key(d):
    ret_str = "{0}/{1}/{2} {3}H".format(d.year,d.month,d.day,d.hour)
    return ret_str

d1 = datetime.datetime.utcnow() - timedelta(days=1)
d2  = datetime.datetime.utcnow()+timedelta(hours=1)
count_twitts_by_range(d1,d2)

# (d2-d1).total_seconds() 

731

In [187]:
def count_hashtags_by_range(start_date,end_date,hashtag):
    delta = get_delta(start_date,end_date)
    h_diff = int(delta.total_seconds()/3600)

    total_count = 0
    for i in range(delta.days * 24 + h_diff):
        date_key = start_date + timedelta (hours=i)
        key = extract_date_key(date_key)+' tag:'+hashtag
        tmp = r.get(key)
        if tmp != None:
            total_count += int(tmp.decode('ascii'))
#         print (extract_date_key(date_str))
    return total_count

def extract_date_key(d):
    ret_str = "{0}/{1}/{2} {3}H".format(d.year,d.month,d.day,d.hour)
    return ret_str
start_date = '2021/07/16'
end_date = '2021/07/17'
# d1 = datetime.datetime.strptime(start_date, '%Y/%m/%d')
d1 = datetime.datetime.utcnow() - timedelta(hours=6)
# d2 = datetime.datetime.strptime(end_date, '%Y/%m/%d')
d2  = datetime.datetime.utcnow()+timedelta(hours=1)
count_hashtags_by_range(d1,d2,'برکت')

47

In [175]:
datetime.datetime.utcnow() - timedelta(hours=6)

datetime.datetime(2021, 7, 16, 8, 46, 12, 264287)

In [ ]:
# from flask import Flask,send_file,make_response
from flask_cors import CORS, cross_origin

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

@app.route('/hashtag_table.html', methods=("POST", "GET"))
def html_table():
    tmp = r.lrange('hashtags_recently_list', 0, -1)
    str1 = b"\n ".join(tmp)
    df = pd.DataFrame(tmp)
    df[0] = df[0].apply(lambda x : x.decode('utf-8'))

    df.rename(columns={0:'هشتگ'},inplace=True)
    return df.to_html(header="true", table_id="table")


@app.route('/twitt_table.html', methods=("POST", "GET"))
def html_twitt_table():
    tmp = r.lrange('twitt_recently_list', 0, -1)
    str1 = b"\n ".join(tmp)
    df = pd.DataFrame(tmp)
    df[0] = df[0].apply(lambda x : x.decode('utf-8'))

    df.rename(columns={0:'محتوای توییت'},inplace=True)
    return df.to_html(header="true", table_id="table")


@app.route('/count_hashtag', methods=("POST", "GET"))
def count_hashtags():
    d1 = datetime.datetime.utcnow()-timedelta(hours=6)
    d2  = datetime.datetime.utcnow()+timedelta(hours=1)
    
    d1_twitts = datetime.datetime.utcnow() - timedelta(days=1)

#     unique_hashtags = 
    
#     count_hashtags_by_range(d1,d2,'برکت')
    tmp_map = {'نماد':['برکت','شستا','خودرو','خساپا','Last 1 Hour Unique Hashtags Count','Twitts Last 1 Hour','ALL Twitt Count Last 6 Hours','ALL Twitt Last 1 Day'],
              'تعداد توییت': [count_hashtags_by_range(d1,d2,'برکت'),count_hashtags_by_range(d1,d2,'شستا'),count_hashtags_by_range(d1,d2,'خودرو'),count_hashtags_by_range(d1,d2,'خساپا')
                              ,count_last_hour_unique_hashtags()
                              ,count_twitts_by_range(d2-timedelta(hours=2),d2),count_twitts_by_range(d1,d2) ,count_twitts_by_range(d1_twitts,d2)]
              }
    return pd.DataFrame(tmp_map).to_html(header="true", table_id="table")




if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jul/2021 10:50:59] "GET /count_hashtag HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:50:59] "GET /twitt_table.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:50:59] "GET /hashtag_table.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51:00] "GET /count_hashtag HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51:00] "GET /twitt_table.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51:00] "GET /hashtag_table.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51:01] "GET /count_hashtag HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51:01] "GET /twitt_table.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51:01] "GET /hashtag_table.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51:32] "GET /count_hashtag HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51:32] "GET /twitt_table.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51:32] "GET /hashtag_table.html HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2021 10:51